In [ ]:
import os
import pandas as pd
import openai
import time
from sklearn.metrics import classification_report

openai.api_key = ''

folder = r"Data\Kfold_HabBe"
folderresults= r"Results_Data\few_shot"
global results1


In [41]:
def process_missing_keys(sentences,missingkeys,results,batch,batch_indices,batchnumber):
    print("i am here in process_missing_keys")
    m={}
    for key,indexlist in missingkeys.items():
        for index in indexlist:
            m[index]=sentences[index]
    
    batch_indices=list(m.keys())
    batch=list(m.values())
    print(batch_indices)
    print(batch)


    def process_batch(batch,batch_indices,batchnumber,results,missingkeys):
       
        print("results in start process_missing_keys function ", results)

        prompt = "\n".join([f"{i}. Classify the sentence '{batch[i]}' as 'habitual be' or 'non-habitual be' in one word and keep the numbering given at the start of the prompt intact in the response:"
                            for i in range(len(batch))])
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            )

            responses_map = {}
            responses = [res.strip() for res in response.choices[0].message['content'].strip().split('\n')]
            print("Actual responses in process_missing_keys ", responses)

            lineindex=0
            for line in responses:
                try:
                    indexr, value = line.split(". ", 1)
                    indexr = int(indexr)
                    responses_map[batch_indices[indexr]] = value
                except ValueError:
                    print(f"Malformed response line : '{line}'")
                    if line.startswith('h') or line.startswith('H') or line.startswith('n') or line.startswith('N'):
                        responses_map[batch_indices[lineindex]]=line
                lineindex+=1

            for i, idx in enumerate(batch_indices):
                if idx in responses_map:
                    results[idx] = responses_map[idx]
                    print("************************************",batchnumber,missingkeys,idx,results[idx])
                    del missingkeys[batchnumber]
                # else:
                #     if batchnumber not in missingkeys:
                #         missingkeys[batchnumber] = []
                #     missingkeys[batchnumber].append(idx)

            print("Responses Map in process_missing_keys ", results)
            print("Missing Keys in process_missing_keys", missingkeys)
            
           

        except openai.error.RateLimitError:
            print("Rate limit exceeded. Waiting to retry...")
            time.sleep(80)  # Wait for 80 seconds before retrying
        # except Exception as e:
        #     print(f"Error processing batch: {e}")
        return results,missingkeys
    return process_batch(batch,batch_indices,batchnumber,results,missingkeys)
        

In [42]:
# Function to generate prompts based on train data
def generate_prompt(train_data):
    examples = []
    for _, row in train_data.head(6).iterrows(): 
        sentence = row['Sentence']
        label = "habitual be" if row['Habituality'] == 1 else "non-habitual be"
        examples.append(f"Sentence: {sentence}\nLabel: {label}\n")
    text="I have given few classified train examples"
    prompt =text+ "\n".join(examples)
    return prompt


In [43]:
def process_batch(prompt, sentences,batch, batch_indices, batchnumber):
    global results1
    missingkeys1={}
    prompt += "\n".join([f"{i}. Classify the sentence '{batch[i]}' as 'habitual be' or 'non-habitual be' in one word and keep the numbering given at the start of the prompt intact in the response:"
                        for i in range(len(batch))])

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )

        responses_map = {}
        responses = [res.strip() for res in response.choices[0].message['content'].strip().split('\n')]
        print("Actual responses: ", responses)
        lineindex=0
        for line in responses:
            try:
                indexr, value = line.split(". ", 1)
                indexr = int(indexr)
                responses_map[batch_indices[indexr]] = value
            except ValueError:
                print(f"Malformed response line : '{line}'")
                print("linindex:  ",lineindex)
                if line.startswith('h') or line.startswith('H') or line.startswith('n') or line.startswith('N'):
                    responses_map[batch_indices[lineindex]]=line
            lineindex+=1
        # print("hererre testtt",responses_map)          
        for i, idx in enumerate(batch_indices):
            if idx in responses_map:
                results1[idx] = responses_map[idx]
            else:
                if batchnumber not in missingkeys1:
                    missingkeys1[batchnumber] = []
                missingkeys1[batchnumber].append(idx)

        # print("Responses Map:", responses_map)
        print("Missing Keys:", missingkeys1)
        print("resultss up",results1)

        if missingkeys1:
            print("did i come here")
            
            newresults,newmissingkeys=process_missing_keys(sentences,missingkeys1,results1,batch,batch_indices,batchnumber)
        # time.sleep(80)      
            missingkeys1=newmissingkeys
            results1=newresults
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Waiting to retry...")
        time.sleep(80)  # Wait for 80 seconds before retrying
    # except Exception as e:
    #     print(f"Error processing batch: {e}")
        time.sleep(80)  # Wait for 80 seconds before retrying
    

    return results1,missingkeys1

In [44]:
# Function to classify sentences in batches
def classify_sentences_batch(prompt,sentences, batch_size=20):
    global results1
    results1 = {i: None for i in range(len(sentences))}  # Initialize results dictionary with None values
    # Process sentences in batches
    batchnumber = 1
    for i in range(0, len(sentences), batch_size):
        batch_indices = list(range(i, min(i + batch_size, len(sentences))))
        localresults,localmissingkeys=process_batch(prompt, sentences,[sentences[idx] for idx in batch_indices], batch_indices, batchnumber)
        # i=1
        # for key,value in localresults.items():
        #     if value==None:
        #         print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ issuesss @@@@@@@@@  :::: ",key, value,batch_indices[key],batchnumber)
        #         i+=1
        # print(i)
        batchnumber += 1
    print("results downnnnnnn******************",results1)
    return results1


In [45]:
import re
def gatherResults(test_data,test_sentences,results):
    print("#################################################################  hcdjcnd")
    fold_predictions = []
    fold_true_labels = []
    for idx in range(len(test_sentences)):
            # Access prediction directly from the results list
            prediction = results[idx] if idx < len(results) else None
            prediction=re.sub(r'^[^a-zA-Z]+', '', prediction)

            if prediction:  # Check if prediction is not None
                prediction = prediction.lower()

            if prediction is None:
                # Handle None values
                numeric_prediction = 0  # Assign a default numeric value  for None predictions
            else:
                # Convert the prediction to numeric: 1 for "habitual be", -1 for "non-habitual be"
                if prediction.startswith('h'):
                    numeric_prediction = 1
                elif prediction.startswith('n'):
                    numeric_prediction = -1
                else:
                    numeric_prediction = 0  # Handle unexpected cases

            # Get the true label
            true_label = 1 if test_data.loc[idx, 'Habituality'] == 1 else -1

            fold_predictions.append(numeric_prediction)
            fold_true_labels.append(true_label)

            # print(f"Sentence {idx}: '{test_sentences[idx]}' -> Classification: {numeric_prediction} (True: {true_label})")
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^",fold_predictions)
    return fold_predictions,fold_true_labels
            


In [46]:
def interleave_labels(positive_data, negative_data):
    """
    Interleave sentences with label `1` and `-1` as much as possible and append remaining.
    """
    interleaved_data = []

    pos_len = len(positive_data)
    neg_len = len(negative_data)
    min_len = min(pos_len, neg_len)

    # Interleave `1`s and `-1`s one by one
    for i in range(min_len):
        interleaved_data.append(positive_data.iloc[i].to_dict())  # Convert row to dictionary
        interleaved_data.append(negative_data.iloc[i].to_dict())  # Convert row to dictionary

    # Append the rest of the remaining data
    if pos_len > neg_len:
        interleaved_data.extend(positive_data.iloc[min_len:].to_dict('records'))
    else:
        interleaved_data.extend(negative_data.iloc[min_len:].to_dict('records'))

    # Convert to DataFrame using the column names explicitly
    interleaved_df = pd.DataFrame(interleaved_data, columns=["Habituality", "Sentence"])
    return interleaved_df


In [47]:
from sklearn.utils import shuffle

completeresults=[]
results_folder = folderresults
def main():
    # Loop through each fold
    train_files = sorted([f for f in os.listdir(folder) if f.startswith("train_sentences_fold") and f.endswith(".csv")])
    test_files = sorted([f for f in os.listdir(folder) if f.startswith("test_sentences_fold") and f.endswith(".csv")])

    all_predictions = []
    all_true_labels = []

    if len(train_files) == len(test_files):
        for fold_idx, (train_file, test_file) in enumerate(zip(train_files, test_files)):
            print(f"Processing {train_file} and {test_file}")
            
            # Load the train and test CSV files
            train_data = pd.read_csv(os.path.join(folder, train_file))
            test_data = pd.read_csv(os.path.join(folder, test_file))
            


            #random shuffling
            ######################################################################
            # test_data = shuffle(test_data, random_state=42)  # Shuffle data with a fixed random seed for reproducibility
            # test_sentences = test_data['Sentence'].tolist()
            #####################################################################
   

            #Shuffling( all 1s and zeros )
            #all 1;s and -1s together
            # #############################################################
            # positive_data = test_data[test_data['Habituality'] == 1]
            # negative_data = test_data[test_data['Habituality'] == -1]

            # grouped_data = pd.concat([positive_data, negative_data], ignore_index=True)

           
            # test_data=grouped_data
            # test_sentences = grouped_data['Sentence'].tolist()
            # true_labels = grouped_data['Habituality'].tolist()
            # print(len(test_sentences))
            #################################################################


            #Shuffling Interleaved data
            ##########################################################################
            positive_data = test_data[test_data['Habituality'] == 1].reset_index(drop=True)
            negative_data =  test_data[test_data['Habituality'] == -1].reset_index(drop=True)

            # Interleave sentences with label 1 and -1
            interleaved_data = interleave_labels(positive_data, negative_data)

            
            test_data=interleaved_data
            test_sentences = interleaved_data['Sentence'].tolist()
            true_labels = interleaved_data['Habituality'].tolist()
            print(len(test_sentences))

            #################################################################################


            #this not yet used(will try later)
            prompt = generate_prompt(train_data.sample(5, random_state=42))  
            test_sentences = test_data['Sentence'].tolist()
            results = classify_sentences_batch(prompt,test_sentences, batch_size=20)
            completeresults.append(results)
            
            fold_predictions,fold_true_labels=gatherResults(test_data,test_sentences,results)
            print(classification_report(fold_true_labels, fold_predictions, target_names=['Non-Habitual (label -1)', 'Habitual (label 1)']))

            print("next &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&",len(fold_predictions),len(fold_true_labels))
            all_predictions.extend(fold_predictions)
            all_true_labels.extend(fold_true_labels)

            # Save the fold results to a CSV file
            output_df = pd.DataFrame({
                'Sentence': test_sentences,
                'True_Label': fold_true_labels,
                'Prediction': fold_predictions
            })
            if fold_idx==2:
                output_file = f"few_shot_interleaved_predictions_Habitual_be_fold_{fold_idx}_predictions.csv"
                print(classification_report(fold_true_labels, fold_predictions, target_names=['Non-Habitual (label -1)', 'Habitual (label 1)']))

                # output_file = os.path.join(results_folder, 'few_shot_full__predictions_Habitual_be.csv')

                # output_file = os.path.join(results_folder, 'few_shot_random_shuffle_predictions_Habitual_be.csv')

                # output_file = os.path.join(results_folder, 'few_shot_New_shuffled(0_and_1_together_predictions_Habitual_be.csv')
                # output_file = os.path.join(results_folder, 'few_shot_New_shuffled_interleaved_predictions_Habitual_be.csv')
                output_df.to_csv(os.path.join(results_folder, output_file), index=False)


                print(f"Results saved to {output_file}")

    
        print("complete prediction set" ,len(all_predictions),"  ",len(all_true_labels))
        return all_predictions,all_true_labels


    # Evaluate across all folds
    print("\nClassification Report across all folds:")

In [48]:
all_predictions=[]
all_true_labels=[]
# all_predictions
all_predictions,all_true_labels=main()


Processing train_sentences_fold_1.csv and test_sentences_fold_1.csv
774
Actual responses:  ['1. non-habitual be', '2. habitual be', '3. non-habitual be', '4. habitual be', '5. non-habitual be', '6. habitual be', '7. habitual be', '8. habitual be', '9. non-habitual be', '10. habitual be', '11. non-habitual be', '12. habitual be', '13. non-habitual be', '14. habitual be', '15. non-habitual be', '16. habitual be', '17. non-habitual be', '18. habitual be', '19. non-habitual be']
Missing Keys: {1: [0]}
resultss up {0: None, 1: 'non-habitual be', 2: 'habitual be', 3: 'non-habitual be', 4: 'habitual be', 5: 'non-habitual be', 6: 'habitual be', 7: 'habitual be', 8: 'habitual be', 9: 'non-habitual be', 10: 'habitual be', 11: 'non-habitual be', 12: 'habitual be', 13: 'non-habitual be', 14: 'habitual be', 15: 'non-habitual be', 16: 'habitual be', 17: 'non-habitual be', 18: 'habitual be', 19: 'non-habitual be', 20: None, 21: None, 22: None, 23: None, 24: None, 25: None, 26: None, 27: None, 28: Non

UnboundLocalError: cannot access local variable 'output_file' where it is not associated with a value

In [28]:
print(len(all_predictions))

print(len(all_true_labels))

7739
7739


In [29]:
# print(all_predictions)
i=1
for index,pred in enumerate(all_predictions):
    if pred==0:
        print(index, pred)
        i+=1
print(i)

1


In [30]:
print(classification_report(all_true_labels, all_predictions, target_names=['Non-Habitual (label -1)', 'Habitual (label 1)']))


                         precision    recall  f1-score   support

Non-Habitual (label -1)       0.83      0.63      0.72      4009
     Habitual (label 1)       0.69      0.86      0.76      3730

               accuracy                           0.74      7739
              macro avg       0.76      0.75      0.74      7739
           weighted avg       0.76      0.74      0.74      7739



In [ ]:
completeresults